We are gonna revese it

In [12]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import torch

# Try to import TabPFN and TabM with fallbacks
try:
    from tabpfn import TabPFNRegressor
    TABPFN_AVAILABLE = True
except ImportError:
    print("⚠️  TabPFN not available. Install with: pip install tabpfn")
    TABPFN_AVAILABLE = False

# TabM proper import - check if it's available and get the correct class
try:
    import tabm
    # Try different possible imports for TabM
    if hasattr(tabm, 'TabMRegressor'):
        TabMRegressor = tabm.TabMRegressor
        TABM_AVAILABLE = True
    elif hasattr(tabm, 'TabM'):
        TabMRegressor = tabm.TabM
        TABM_AVAILABLE = True
    else:
        # Fallback: inspect the module to find available classes
        available_attrs = [attr for attr in dir(tabm) if not attr.startswith('_')]
        print(f"⚠️  TabM available attributes: {available_attrs}")
        TABM_AVAILABLE = False
except ImportError:
    print("⚠️  TabM not available. Install from GitHub: pip install git+https://github.com/yandex-research/tabm.git")
    TABM_AVAILABLE = False

print("Loading data...")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train.drop([f'BlendProperty{i}' for i in range(1, 11)], axis=1)
y = train[[f'BlendProperty{i}' for i in range(1, 11)]]
X_test = test.drop(['ID'], axis=1)

kf = KFold(n_splits=5, shuffle=True)

print("🚀 Starting Level 1 training...")

# Level 1: Advanced models as base models - ALWAYS include XGBoost
level1_names = ['XGB']  # XGBoost is always included in Level 1

# Add available advanced models to level 1
if TABM_AVAILABLE:
    level1_names.append('TabM')
if TABPFN_AVAILABLE:
    level1_names.append('TabPFN')

# Add additional strong models to Level 1 for better diversity
level1_names.extend(['LGBM', 'RandomForest', 'ExtraTrees'])

print(f"📊 Level 1 models: {', '.join(level1_names)}")

level1_oof = {name: np.zeros(y.shape) for name in level1_names}
level1_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level1_names}

for t in range(y.shape[1]):
    print(f"🎯 Training Level 1 for target BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        # XGBoost - ALWAYS included with optimized parameters
        xgb_model = xgb.XGBRegressor(
            n_estimators=200,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            verbosity=0,
            n_jobs=-1
        )
        xgb_model.fit(X_tr, y_tr)
        level1_oof['XGB'][val_idx, t] = xgb_model.predict(X_val)
        level1_test['XGB'][:, t] += xgb_model.predict(X_test) / kf.n_splits

        # LightGBM with optimized parameters
        lgb_model = lgb.LGBMRegressor(
            n_estimators=200,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            verbose=-1,
            n_jobs=-1
        )
        lgb_model.fit(X_tr, y_tr)
        level1_oof['LGBM'][val_idx, t] = lgb_model.predict(X_val)
        level1_test['LGBM'][:, t] += lgb_model.predict(X_test) / kf.n_splits

        # RandomForest with increased estimators
        rf = RandomForestRegressor(
            n_estimators=200,
            max_depth=10,
            n_jobs=-1
        )
        rf.fit(X_tr, y_tr)
        level1_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
        level1_test['RandomForest'][:, t] += rf.predict(X_test) / kf.n_splits

        # ExtraTrees with increased estimators
        et = ExtraTreesRegressor(
            n_estimators=200,
            max_depth=10,
            n_jobs=-1
        )
        et.fit(X_tr, y_tr)
        level1_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
        level1_test['ExtraTrees'][:, t] += et.predict(X_test) / kf.n_splits

        # TabM (if available)
        if 'TabM' in level1_names and TABM_AVAILABLE:
            try:
                # Create TabM model - adjust parameters as needed
                tabm_model = TabMRegressor(
                    device='cuda' if torch.cuda.is_available() else 'cpu'
                )
                tabm_model.fit(X_tr.values, y_tr.values)
                level1_oof['TabM'][val_idx, t] = tabm_model.predict(X_val.values)
                level1_test['TabM'][:, t] += tabm_model.predict(X_test.values) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabM: {e}")
                # Remove TabM from level1_names if it fails
                if 'TabM' in level1_names:
                    level1_names.remove('TabM')

        # TabPFN (if available)
        if 'TabPFN' in level1_names and TABPFN_AVAILABLE:
            try:
                tabpfn = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                tabpfn.fit(X_tr, y_tr.values)
                level1_oof['TabPFN'][val_idx, t] = tabpfn.predict(X_val)
                level1_test['TabPFN'][:, t] += tabpfn.predict(X_test) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabPFN: {e}")
                if 'TabPFN' in level1_names:
                    level1_names.remove('TabPFN')

# Remove empty entries from level1 dictionaries
level1_names = [name for name in level1_names if name in level1_oof and level1_oof[name].sum() != 0]
level1_oof = {name: level1_oof[name] for name in level1_names}
level1_test = {name: level1_test[name] for name in level1_names}

print(f"\n📊 Level 1 MAPE Scores (using {len(level1_names)} models):")
for name in level1_names:
    mape = mean_absolute_percentage_error(y, level1_oof[name])
    print(f"  {name}: {mape:.6f}")

print("\n🔄 Preparing Level 2 inputs from Level 1 outputs...")
stack_X = np.concatenate([level1_oof[name] for name in level1_names], axis=1)
stack_X_test = np.concatenate([level1_test[name] for name in level1_names], axis=1)

print("\n🚀 Starting Level 2 stacking (Traditional ML Models)...")

# Level 2: Traditional ML models as stacking models
level2_names = [
    'Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'Huber',
    'RandomForest_L2', 'ExtraTrees_L2', 'AdaBoost', 'GradientBoost', 'Bagging',
    'KNN', 'SVR', 'XGB_L2', 'LGBM_L2'  # Added L2 suffix to avoid confusion
]
level2_oof = {name: np.zeros(y.shape) for name in level2_names}
level2_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level2_names}

for t in range(y.shape[1]):
    print(f"🎯 Level 2 stacking for BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(stack_X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = stack_X[tr_idx], stack_X[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        # Ridge
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_tr, y_tr)
        level2_oof['Ridge'][val_idx, t] = ridge.predict(X_val)
        level2_test['Ridge'][:, t] += ridge.predict(stack_X_test) / kf.n_splits

        # Lasso
        lasso = Lasso(alpha=0.1, max_iter=1000)
        lasso.fit(X_tr, y_tr)
        level2_oof['Lasso'][val_idx, t] = lasso.predict(X_val)
        level2_test['Lasso'][:, t] += lasso.predict(stack_X_test) / kf.n_splits

        # ElasticNet
        elastic = ElasticNet(alpha=0.1, max_iter=1000)
        elastic.fit(X_tr, y_tr)
        level2_oof['ElasticNet'][val_idx, t] = elastic.predict(X_val)
        level2_test['ElasticNet'][:, t] += elastic.predict(stack_X_test) / kf.n_splits

        # BayesianRidge
        bayesian = BayesianRidge()
        bayesian.fit(X_tr, y_tr)
        level2_oof['BayesianRidge'][val_idx, t] = bayesian.predict(X_val)
        level2_test['BayesianRidge'][:, t] += bayesian.predict(stack_X_test) / kf.n_splits

        # Huber
        huber = HuberRegressor()
        huber.fit(X_tr, y_tr)
        level2_oof['Huber'][val_idx, t] = huber.predict(X_val)
        level2_test['Huber'][:, t] += huber.predict(stack_X_test) / kf.n_splits

        # RandomForest (Level 2)
        rf = RandomForestRegressor(n_estimators=50)
        rf.fit(X_tr, y_tr)
        level2_oof['RandomForest_L2'][val_idx, t] = rf.predict(X_val)
        level2_test['RandomForest_L2'][:, t] += rf.predict(stack_X_test) / kf.n_splits

        # ExtraTrees (Level 2)
        et = ExtraTreesRegressor(n_estimators=50)
        et.fit(X_tr, y_tr)
        level2_oof['ExtraTrees_L2'][val_idx, t] = et.predict(X_val)
        level2_test['ExtraTrees_L2'][:, t] += et.predict(stack_X_test) / kf.n_splits

        # AdaBoost
        ada = AdaBoostRegressor(n_estimators=50)
        ada.fit(X_tr, y_tr)
        level2_oof['AdaBoost'][val_idx, t] = ada.predict(X_val)
        level2_test['AdaBoost'][:, t] += ada.predict(stack_X_test) / kf.n_splits

        # GradientBoosting
        gb = GradientBoostingRegressor(n_estimators=100)
        gb.fit(X_tr, y_tr)
        level2_oof['GradientBoost'][val_idx, t] = gb.predict(X_val)
        level2_test['GradientBoost'][:, t] += gb.predict(stack_X_test) / kf.n_splits

        # Bagging
        bag = BaggingRegressor(n_estimators=50)
        bag.fit(X_tr, y_tr)
        level2_oof['Bagging'][val_idx, t] = bag.predict(X_val)
        level2_test['Bagging'][:, t] += bag.predict(stack_X_test) / kf.n_splits

        # KNN
        knn = KNeighborsRegressor(n_neighbors=5)
        knn.fit(X_tr, y_tr)
        level2_oof['KNN'][val_idx, t] = knn.predict(X_val)
        level2_test['KNN'][:, t] += knn.predict(stack_X_test) / kf.n_splits

        # SVR
        svr = SVR(kernel='rbf', gamma='scale')
        svr.fit(X_tr, y_tr)
        level2_oof['SVR'][val_idx, t] = svr.predict(X_val)
        level2_test['SVR'][:, t] += svr.predict(stack_X_test) / kf.n_splits

        # XGBoost (Level 2)
        xgb_model = xgb.XGBRegressor(verbosity=0)
        xgb_model.fit(X_tr, y_tr)
        level2_oof['XGB_L2'][val_idx, t] = xgb_model.predict(X_val)
        level2_test['XGB_L2'][:, t] += xgb_model.predict(stack_X_test) / kf.n_splits

        # LightGBM (Level 2)
        lgb_model = lgb.LGBMRegressor(verbose=-1)
        lgb_model.fit(X_tr, y_tr)
        level2_oof['LGBM_L2'][val_idx, t] = lgb_model.predict(X_val)
        level2_test['LGBM_L2'][:, t] += lgb_model.predict(stack_X_test) / kf.n_splits

print("\n📊 Level 2 MAPE Scores:")
level2_scores = {}
for name in level2_names:
    mape = mean_absolute_percentage_error(y, level2_oof[name])
    level2_scores[name] = mape
    print(f"  {name}: {mape:.6f}")

print("\n🔍 Starting Dynamic Level 3 optimization...")

# Dynamic selection of top performers (configurable top K models)
TOP_K = min(6, len(level2_names))  # Select top 6 models or all if less than 6
sorted_models = sorted(level2_scores.items(), key=lambda x: x[1])
top_models = [model[0] for model in sorted_models[:TOP_K]]

print(f"\n🎯 Selected top {len(top_models)} models for Level 3:")
for i, model in enumerate(top_models):
    print(f"  {i+1}. {model}: {level2_scores[model]:.6f}")

# Prepare Level 3 inputs with top performers
level3_oof = np.concatenate([level2_oof[model] for model in top_models], axis=1)
level3_test = np.concatenate([level2_test[model] for model in top_models], axis=1)

def objective(trial):
    # Dynamically suggest weights for each top model
    weights = {}
    for model in top_models:
        weights[model] = trial.suggest_float(f'w_{model}', 0.0, 1.0)
    
    # Normalize weights
    total_weight = sum(weights.values())
    if total_weight == 0:
        return float('inf')
    
    normalized_weights = {k: v/total_weight for k, v in weights.items()}
    
    # Create weighted ensemble
    ensemble_pred = sum(normalized_weights[model] * level2_oof[model] 
                       for model in top_models)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y, ensemble_pred)
    return mape

# Optimize weights
print("🔧 Optimizing ensemble weights...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

print(f"\n✅ Best MAPE: {study.best_value:.6f}")
print("🎯 Best weights:")
best_params = study.best_params
for param, value in best_params.items():
    print(f"  {param}: {value:.4f}")

# Normalize best weights
total_weight = sum(best_params.values())
normalized_weights = {k: v/total_weight for k, v in best_params.items()}

print("\n📊 Normalized weights:")
for param, value in normalized_weights.items():
    print(f"  {param}: {value:.4f}")

# Create final ensemble predictions
final_test = sum(normalized_weights[f'w_{model}'] * level2_test[model] 
                 for model in top_models)

# Final validation score
final_oof = sum(normalized_weights[f'w_{model}'] * level2_oof[model] 
                for model in top_models)

final_mape = mean_absolute_percentage_error(y, final_oof)
print(f"\n🎉 Final ensemble MAPE: {final_mape:.6f}")

print("\n📊 Individual target MAPE scores:")
for i in range(y.shape[1]):
    target_mape = mean_absolute_percentage_error(y.iloc[:, i], final_oof[:, i])
    print(f"  BlendProperty{i+1}: {target_mape:.6f}")

# Save submission with descriptive filename
submission = pd.DataFrame(final_test, columns=[f'BlendProperty{i}' for i in range(1, 11)])
submission.insert(0, 'ID', test['ID'])
submission_filename = f"submission_xgb_level1_ensemble_top{TOP_K}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n💾 Submission file saved as '{submission_filename}'")

print(f"\n🎯 Advanced Dynamic Ensemble Summary:")
print(f"  Level 1: {len(level1_names)} strong models including XGBoost: {', '.join(level1_names)}")
print(f"  Level 2: {len(level2_names)} traditional ML models stacking on Level 1 outputs") 
print(f"  Level 3: Dynamic optimization with top {len(top_models)} performers: {', '.join(top_models)}")
print(f"  Final MAPE: {final_mape:.6f}")
print(f"  Submission saved as: {submission_filename}")

# Instructions for installing missing packages
print("\n📝 Installation Instructions:")
if not TABM_AVAILABLE:
    print("  For TabM: pip install git+https://github.com/yandex-research/tabm.git")
if not TABPFN_AVAILABLE:
    print("  For TabPFN: pip install tabpfn")

Loading data...
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
  Fold 1/5
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty2...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty3...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty4...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty5...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty6...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty7...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty8...
  Fold 1/5
  Fold 2/5
  Fold 

[I 2025-07-23 09:43:26,832] A new study created in memory with name: no-name-69bffc49-f779-4038-82ea-94eacf84f2f6



📊 Level 2 MAPE Scores:
  Ridge: 0.515569
  Lasso: 0.528366
  ElasticNet: 0.636717
  BayesianRidge: 0.517427
  Huber: 0.537089
  RandomForest_L2: 0.549829
  ExtraTrees_L2: 0.564001
  AdaBoost: 0.839170
  GradientBoost: 0.540827
  Bagging: 0.588813
  KNN: 2.394996
  SVR: 0.636229
  XGB_L2: 0.573953
  LGBM_L2: 0.580093

🔍 Starting Dynamic Level 3 optimization...

🎯 Selected top 6 models for Level 3:
  1. Ridge: 0.515569
  2. BayesianRidge: 0.517427
  3. Lasso: 0.528366
  4. Huber: 0.537089
  5. GradientBoost: 0.540827
  6. RandomForest_L2: 0.549829
🔧 Optimizing ensemble weights...


Best trial: 4. Best value: 0.499434:   5%|▌         | 10/200 [00:00<00:01, 95.10it/s]

[I 2025-07-23 09:43:26,874] Trial 0 finished with value: 0.5096653983344902 and parameters: {'w_Ridge': 0.3777470641206069, 'w_BayesianRidge': 0.06731797475200196, 'w_Lasso': 0.09583469588460769, 'w_Huber': 0.9759376037490679, 'w_GradientBoost': 0.9350685961064421, 'w_RandomForest_L2': 0.11089082794374472}. Best is trial 0 with value: 0.5096653983344902.
[I 2025-07-23 09:43:26,884] Trial 1 finished with value: 0.5020266642965664 and parameters: {'w_Ridge': 0.12003264767236921, 'w_BayesianRidge': 0.800742396896894, 'w_Lasso': 0.834121103289503, 'w_Huber': 0.6286495547476066, 'w_GradientBoost': 0.6585025551549453, 'w_RandomForest_L2': 0.16876551157098552}. Best is trial 1 with value: 0.5020266642965664.
[I 2025-07-23 09:43:26,893] Trial 2 finished with value: 0.5074593236577419 and parameters: {'w_Ridge': 0.33164687209016774, 'w_BayesianRidge': 0.6881483159794258, 'w_Lasso': 0.5968503350217302, 'w_Huber': 0.806563719526527, 'w_GradientBoost': 0.31229549914666543, 'w_RandomForest_L2': 0.1

...

[I 2025-07-23 09:43:34,600] Trial 187 finished with value: 0.4947894426719096 and parameters: {'w_Ridge': 0.6404811440501409, 'w_BayesianRidge': 0.4427365627792294, 'w_Lasso': 0.5360220381879317, 'w_Huber': 0.03628305760402932, 'w_GradientBoost': 0.41514660177950874, 'w_RandomForest_L2': 0.5705796316110434}. Best is trial 183 with value: 0.4942624982354629.
[I 2025-07-23 09:43:34,649] Trial 188 finished with value: 0.4943272151640777 and parameters: {'w_Ridge': 0.6805603841809486, 'w_BayesianRidge': 0.3963867529852035, 'w_Lasso': 0.5180373608700557, 'w_Huber': 0.001023124920971237, 'w_GradientBoost': 0.405581247758883, 'w_RandomForest_L2': 0.6200847659031788}. Best is trial 183 with value: 0.4942624982354629.
[I 2025-07-23 09:43:34,683] Trial 189 finished with value: 0.494258000503916 and parameters: {'w_Ridge': 0.6177549652121137, 'w_BayesianRidge': 0.4088901796225676, 'w_Lasso': 0.5174544450692309, 'w_Huber': 0.0002704796661375376, 'w_GradientBoost': 0.4363799894679422, 'w_RandomFore

Best trial: 189. Best value: 0.494258:  98%|█████████▊| 195/200 [00:08<00:00, 22.52it/s]

[I 2025-07-23 09:43:34,821] Trial 192 finished with value: 0.49436565166915586 and parameters: {'w_Ridge': 0.6579167870656827, 'w_BayesianRidge': 0.3943113485197928, 'w_Lasso': 0.4860448602763962, 'w_Huber': 0.006477652341374698, 'w_GradientBoost': 0.4656684217504305, 'w_RandomForest_L2': 0.6123507776178693}. Best is trial 189 with value: 0.494258000503916.
[I 2025-07-23 09:43:34,865] Trial 193 finished with value: 0.49444729428500345 and parameters: {'w_Ridge': 0.691320753571518, 'w_BayesianRidge': 0.3992802018303711, 'w_Lasso': 0.4854280631664657, 'w_Huber': 0.003960237306160624, 'w_GradientBoost': 0.5084048866556777, 'w_RandomForest_L2': 0.560656430462411}. Best is trial 189 with value: 0.494258000503916.
[I 2025-07-23 09:43:34,907] Trial 194 finished with value: 0.4943259734821851 and parameters: {'w_Ridge': 0.6593936868738851, 'w_BayesianRidge': 0.40522446962371794, 'w_Lasso': 0.5031568156452034, 'w_Huber': 0.0014790863512325137, 'w_GradientBoost': 0.4677961189605116, 'w_RandomFor

Best trial: 189. Best value: 0.494258: 100%|██████████| 200/200 [00:08<00:00, 24.03it/s]


[I 2025-07-23 09:43:34,999] Trial 196 finished with value: 0.4942901825615282 and parameters: {'w_Ridge': 0.6560913737551578, 'w_BayesianRidge': 0.35104214796485206, 'w_Lasso': 0.4689579219785603, 'w_Huber': 0.0009387803766608769, 'w_GradientBoost': 0.46144710669337424, 'w_RandomForest_L2': 0.5831439415101272}. Best is trial 189 with value: 0.494258000503916.
[I 2025-07-23 09:43:35,059] Trial 197 finished with value: 0.4942964282349654 and parameters: {'w_Ridge': 0.6559038645447149, 'w_BayesianRidge': 0.3563651049649467, 'w_Lasso': 0.46738243985390054, 'w_Huber': 0.0016694853101510382, 'w_GradientBoost': 0.4695562422027543, 'w_RandomForest_L2': 0.5923902469295373}. Best is trial 189 with value: 0.494258000503916.
[I 2025-07-23 09:43:35,120] Trial 198 finished with value: 0.494736951057502 and parameters: {'w_Ridge': 0.6595975960859178, 'w_BayesianRidge': 0.3428791460517311, 'w_Lasso': 0.46368591098112005, 'w_Huber': 0.04487768027937605, 'w_GradientBoost': 0.4838068414860288, 'w_RandomF

infinite loop

In [10]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import torch
from datetime import datetime

# Try to import TabPFN and TabM with fallbacks
try:
    from tabpfn import TabPFNRegressor
    TABPFN_AVAILABLE = True
except ImportError:
    print("⚠️  TabPFN not available. Install with: pip install tabpfn")
    TABPFN_AVAILABLE = False

try:
    import tabm
    if hasattr(tabm, 'TabMRegressor'):
        TabMRegressor = tabm.TabMRegressor
        TABM_AVAILABLE = True
    elif hasattr(tabm, 'TabM'):
        TabMRegressor = tabm.TabM
        TABM_AVAILABLE = True
    else:
        available_attrs = [attr for attr in dir(tabm) if not attr.startswith('_')]
        print(f"⚠️  TabM available attributes: {available_attrs}")
        TABM_AVAILABLE = False
except ImportError:
    print("⚠️  TabM not available. Install from GitHub: pip install git+https://github.com/yandex-research/tabm.git")
    TABM_AVAILABLE = False

# Create output folder if it doesn't exist
os.makedirs('submissions', exist_ok=True)

# Create log CSV if it doesn't exist
log_file = 'submissions/ensemble_log.csv'
if not os.path.exists(log_file):
    pd.DataFrame(columns=['timestamp', 'submission', 'final_mape', 'top_models', 'weights']).to_csv(log_file, index=False)

run_number = 1
while True:
    print(f"\n🚀 🚀 🚀 Starting new run #{run_number} 🚀 🚀 🚀\n")

    print("Loading data...")
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    X = train.drop([f'BlendProperty{i}' for i in range(1, 11)], axis=1)
    y = train[[f'BlendProperty{i}' for i in range(1, 11)]]
    X_test = test.drop(['ID'], axis=1)

    kf = KFold(n_splits=5, shuffle=True)

    print("🚀 Starting Level 1 training...")

    level1_names = ['XGB']
    if TABM_AVAILABLE:
        level1_names.append('TabM')
    if TABPFN_AVAILABLE:
        level1_names.append('TabPFN')
    level1_names.extend(['LGBM', 'RandomForest', 'ExtraTrees'])

    print(f"📊 Level 1 models: {', '.join(level1_names)}")

    level1_oof = {name: np.zeros(y.shape) for name in level1_names}
    level1_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level1_names}

    for t in range(y.shape[1]):
        print(f"🎯 Training Level 1 for target BlendProperty{t+1}...")
        for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
            print(f"  Fold {fold+1}/5")
            X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

            xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, verbosity=0, n_jobs=-1)
            xgb_model.fit(X_tr, y_tr)
            level1_oof['XGB'][val_idx, t] = xgb_model.predict(X_val)
            level1_test['XGB'][:, t] += xgb_model.predict(X_test) / kf.n_splits

            lgb_model = lgb.LGBMRegressor(n_estimators=200, max_depth=6, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, verbose=-1, n_jobs=-1)
            lgb_model.fit(X_tr, y_tr)
            level1_oof['LGBM'][val_idx, t] = lgb_model.predict(X_val)
            level1_test['LGBM'][:, t] += lgb_model.predict(X_test) / kf.n_splits

            rf = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1)
            rf.fit(X_tr, y_tr)
            level1_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
            level1_test['RandomForest'][:, t] += rf.predict(X_test) / kf.n_splits

            et = ExtraTreesRegressor(n_estimators=200, max_depth=10, n_jobs=-1)
            et.fit(X_tr, y_tr)
            level1_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
            level1_test['ExtraTrees'][:, t] += et.predict(X_test) / kf.n_splits

            if 'TabM' in level1_names and TABM_AVAILABLE:
                try:
                    tabm_model = TabMRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                    tabm_model.fit(X_tr.values, y_tr.values)
                    level1_oof['TabM'][val_idx, t] = tabm_model.predict(X_val.values)
                    level1_test['TabM'][:, t] += tabm_model.predict(X_test.values) / kf.n_splits
                except Exception as e:
                    print(f"    Error with TabM: {e}")
                    level1_names.remove('TabM')

            if 'TabPFN' in level1_names and TABPFN_AVAILABLE:
                try:
                    tabpfn = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                    tabpfn.fit(X_tr, y_tr.values)
                    level1_oof['TabPFN'][val_idx, t] = tabpfn.predict(X_val)
                    level1_test['TabPFN'][:, t] += tabpfn.predict(X_test) / kf.n_splits
                except Exception as e:
                    print(f"    Error with TabPFN: {e}")
                    level1_names.remove('TabPFN')

    level1_names = [name for name in level1_names if name in level1_oof and level1_oof[name].sum() != 0]
    level1_oof = {name: level1_oof[name] for name in level1_names}
    level1_test = {name: level1_test[name] for name in level1_names}

    print("\n🔄 Preparing Level 2 inputs...")
    stack_X = np.concatenate([level1_oof[name] for name in level1_names], axis=1)
    stack_X_test = np.concatenate([level1_test[name] for name in level1_names], axis=1)

    level2_names = ['Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'Huber', 'RandomForest_L2', 'ExtraTrees_L2', 'AdaBoost', 'GradientBoost', 'Bagging', 'KNN', 'SVR', 'XGB_L2', 'LGBM_L2']
    level2_oof = {name: np.zeros(y.shape) for name in level2_names}
    level2_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level2_names}

    for t in range(y.shape[1]):
        for fold, (tr_idx, val_idx) in enumerate(kf.split(stack_X)):
            X_tr, X_val = stack_X[tr_idx], stack_X[val_idx]
            y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

            Ridge(alpha=1.0).fit(X_tr, y_tr)
            level2_oof['Ridge'][val_idx, t] = Ridge(alpha=1.0).fit(X_tr, y_tr).predict(X_val)

            lasso = Lasso(alpha=0.1).fit(X_tr, y_tr)
            level2_oof['Lasso'][val_idx, t] = lasso.predict(X_val)

            elastic = ElasticNet(alpha=0.1).fit(X_tr, y_tr)
            level2_oof['ElasticNet'][val_idx, t] = elastic.predict(X_val)

            bayesian = BayesianRidge().fit(X_tr, y_tr)
            level2_oof['BayesianRidge'][val_idx, t] = bayesian.predict(X_val)

            huber = HuberRegressor().fit(X_tr, y_tr)
            level2_oof['Huber'][val_idx, t] = huber.predict(X_val)

            rf = RandomForestRegressor(n_estimators=50).fit(X_tr, y_tr)
            level2_oof['RandomForest_L2'][val_idx, t] = rf.predict(X_val)

            et = ExtraTreesRegressor(n_estimators=50).fit(X_tr, y_tr)
            level2_oof['ExtraTrees_L2'][val_idx, t] = et.predict(X_val)

            ada = AdaBoostRegressor(n_estimators=50).fit(X_tr, y_tr)
            level2_oof['AdaBoost'][val_idx, t] = ada.predict(X_val)

            gb = GradientBoostingRegressor(n_estimators=100).fit(X_tr, y_tr)
            level2_oof['GradientBoost'][val_idx, t] = gb.predict(X_val)

            bag = BaggingRegressor(n_estimators=50).fit(X_tr, y_tr)
            level2_oof['Bagging'][val_idx, t] = bag.predict(X_val)

            knn = KNeighborsRegressor(n_neighbors=5).fit(X_tr, y_tr)
            level2_oof['KNN'][val_idx, t] = knn.predict(X_val)

            svr = SVR().fit(X_tr, y_tr)
            level2_oof['SVR'][val_idx, t] = svr.predict(X_val)

            xgb_model = xgb.XGBRegressor().fit(X_tr, y_tr)
            level2_oof['XGB_L2'][val_idx, t] = xgb_model.predict(X_val)

            lgb_model = lgb.LGBMRegressor().fit(X_tr, y_tr)
            level2_oof['LGBM_L2'][val_idx, t] = lgb_model.predict(X_val)

    level2_scores = {name: mean_absolute_percentage_error(y, level2_oof[name]) for name in level2_names}
    top_models = sorted(level2_scores, key=level2_scores.get)[:6]

    level3_oof = np.concatenate([level2_oof[name] for name in top_models], axis=1)
    level3_test = np.concatenate([level2_test[name] for name in top_models], axis=1)

    def objective(trial):
        weights = np.array([trial.suggest_float(f'w_{m}', 0.0, 1.0) for m in top_models])
        if weights.sum() == 0: return 1e6
        weights /= weights.sum()
        blended = sum(weights[i] * level2_oof[m] for i, m in enumerate(top_models))
        return mean_absolute_percentage_error(y, blended)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_weights = np.array([study.best_params[f'w_{m}'] for m in top_models])
    best_weights /= best_weights.sum()

    final_pred = sum(best_weights[i] * level2_test[m] for i, m in enumerate(top_models))
    final_mape = mean_absolute_percentage_error(y, sum(best_weights[i] * level2_oof[m] for i, m in enumerate(top_models)))

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    submission_file = f'submissions/submission_{timestamp}_run{run_number}.csv'
    submission = pd.DataFrame(final_pred, columns=[f'BlendProperty{i}' for i in range(1, 11)])
    submission.insert(0, 'ID', test['ID'])
    submission.to_csv(submission_file, index=False)

    print(f"\n✅ Run #{run_number} complete! Final MAPE: {final_mape:.6f}")
    print(f"Submission saved as: {submission_file}")

    log_df = pd.read_csv(log_file)
    new_row = pd.DataFrame([{
        'timestamp': timestamp,
        'submission': submission_file,
        'final_mape': final_mape,
        'top_models': '|'.join(top_models),
        'weights': '|'.join(f"{w:.4f}" for w in best_weights)
    }])
    log_df = pd.concat([log_df, new_row], ignore_index=True)
    log_df.to_csv(log_file, index=False)


    run_number += 1



🚀 🚀 🚀 Starting new run #1 🚀 🚀 🚀

Loading data...
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
  Fold 1/5


KeyboardInterrupt: 